In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ml-w13p2/train.csv
/kaggle/input/2022-ml-w13p2/test.csv
/kaggle/input/2022-ml-w13p2/sample_submit.csv


In [2]:
submit = pd.read_csv('../input/2022-ml-w13p2/sample_submit.csv')
test = pd.read_csv('../input/2022-ml-w13p2/test.csv')
train = pd.read_csv('../input/2022-ml-w13p2/train.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4788 entries, 0 to 4787
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             4788 non-null   int64  
 1   Unnamed: 0        4788 non-null   int64  
 2   customerID        4788 non-null   object 
 3   gender            4788 non-null   object 
 4   SeniorCitizen     4788 non-null   int64  
 5   Partner           4788 non-null   object 
 6   Dependents        4788 non-null   object 
 7   tenure            4788 non-null   int64  
 8   PhoneService      4788 non-null   object 
 9   MultipleLines     4788 non-null   object 
 10  InternetService   4788 non-null   object 
 11  OnlineSecurity    4788 non-null   object 
 12  OnlineBackup      4788 non-null   object 
 13  DeviceProtection  4788 non-null   object 
 14  TechSupport       4788 non-null   object 
 15  StreamingTV       4788 non-null   object 
 16  StreamingMovies   4788 non-null   object 


In [4]:
temp = train['Churn'].copy()
temp[temp == 'Yes'] = 1
temp[temp == 'No'] = 0
train['Churn'] = temp.astype('int64')

In [5]:
x_train = train.drop(['index', 'Unnamed: 0', 'customerID', 'Churn'], axis=1)
y_train = train['Churn']
x_test = test.drop(['index', 'Unnamed: 0', 'customerID'], axis=1)

In [6]:
# info를 보고 object인 type은 unique()를 통해 내용물을 확인하자
x_train.info()
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4788 entries, 0 to 4787
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            4788 non-null   object 
 1   SeniorCitizen     4788 non-null   int64  
 2   Partner           4788 non-null   object 
 3   Dependents        4788 non-null   object 
 4   tenure            4788 non-null   int64  
 5   PhoneService      4788 non-null   object 
 6   MultipleLines     4788 non-null   object 
 7   InternetService   4788 non-null   object 
 8   OnlineSecurity    4788 non-null   object 
 9   OnlineBackup      4788 non-null   object 
 10  DeviceProtection  4788 non-null   object 
 11  TechSupport       4788 non-null   object 
 12  StreamingTV       4788 non-null   object 
 13  StreamingMovies   4788 non-null   object 
 14  Contract          4788 non-null   object 
 15  PaperlessBilling  4788 non-null   object 
 16  PaymentMethod     4788 non-null   object 


In [7]:
# 복잡하게 여러번 쓸 필요없이 배열 만들어서 수행하면 편리하게 가넝 ~
# 전체 주석 :ctrl + /

columns = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
           'InternetService','OnlineSecurity','OnlineBackup','DeviceProtection',
           'TechSupport','StreamingTV','StreamingMovies','Contract',
           'PaperlessBilling','PaymentMethod']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

x_data = pd.concat([x_train, x_test], axis=0)

for column in columns:
    le.fit(x_data[column])
    x_train[column] = le.fit_transform(x_train[column])
    x_test[column] = le.fit_transform(x_test[column])
    

In [8]:
y_train

0       0
1       0
2       1
3       0
4       0
       ..
4783    1
4784    0
4785    0
4786    1
4787    0
Name: Churn, Length: 4788, dtype: int64

In [9]:
# TotalCharges 형변환 시도 했으나 오류 발견 !
# x_train['TotalCharges'] = x_train['TotalCharges'].astype('float')
# x_train['TotalCharges'] = pd.to_numeric(x_train['TotalCharges'], downcast='float')

In [10]:
x_train['TotalCharges'][280]

' '

In [11]:
(x_train['TotalCharges'] == ' ').sum()

# 갯수가 적을 때는 drop하는 것이 좋은 방법 ~
drop_ipx  = x_train[x_train['TotalCharges'] == ' '].index

x_train = x_train.drop(drop_ipx, axis=0)
y_train = y_train.drop(drop_ipx, axis=0)

In [12]:
x_train['TotalCharges'] = x_train['TotalCharges'].astype('float')

In [13]:
from sklearn.linear_model import LogisticRegression

#clf = LogisticRegression()
# max_iter = 100 (default) : 수렴 안되었다는 메시지 -> 그럼 max_iter 증가 시켜줘야함

clf = LogisticRegression(max_iter=1000)
clf.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [14]:
submit

,index,Churn
0,5,NaN
1,10,NaN
2,13,NaN
3,18,NaN
4,20,NaN
...,...,...
1193,5962,NaN
1194,5968,NaN
1195,5975,NaN
1196,5977,NaN


In [15]:
submit['Churn'] = clf.predict(x_test)

In [16]:
submit.to_csv('result.csv', index=False)